In [1]:
import ccxt
import pandas as pd
import datetime
import time

def fetch_ohlcv_data(symbol, timeframe, since, exchange_id='binance'):
    """
    Скачивает исторические данные (OHLCV) с заданной биржи.

    Args:
        symbol (str): Торговая пара (например, 'BTC/USDT').
        timeframe (str): Временной интервал (например, '1h').
        since (int): Начальное время в миллисекундах.
        exchange_id (str): ID биржи ('binance' по умолчанию).

    Returns:
        pd.DataFrame: DataFrame с данными OHLCV.
    """
    try:
        # Инициализация биржи
        exchange = getattr(ccxt, exchange_id)()
        
        # Проверка, поддерживает ли биржа запрашиваемый таймфрейм
        if timeframe not in exchange.timeframes:
            print(f"Биржа {exchange_id} не поддерживает таймфрейм '{timeframe}'.")
            return None
            
        print(f"Скачиваем данные для {symbol} с биржи {exchange_id}...")
        
        all_ohlcv = []
        
        # Загрузка данных частями, так как биржи ограничивают количество свечей за один запрос
        while True:
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since, limit=1000)
            if not ohlcv:
                break
            
            all_ohlcv.extend(ohlcv)
            # Обновляем 'since' для следующего запроса, чтобы не скачивать дубликаты
            since = all_ohlcv[-1][0] + exchange.parse_timeframe(timeframe) * 1000
            
            # Небольшая задержка, чтобы не превысить лимиты запросов
            time.sleep(exchange.rateLimit / 1000)
            
            # Проверяем, достигнута ли текущая дата
            if datetime.datetime.fromtimestamp(since / 1000) > datetime.datetime.now():
                break

    except (ccxt.NetworkError, ccxt.ExchangeError) as e:
        print(f"Ошибка при подключении к бирже: {e}")
        return None
    
    # Преобразование данных в DataFrame
    df = pd.DataFrame(all_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    
    return df

if __name__ == "__main__":
    # Параметры запроса
    symbol = 'BTC/USDT'
    timeframe = '1h'
    # Рассчитываем дату начала - 1 год назад
    one_year_ago = datetime.datetime.now() - datetime.timedelta(days=365*3)
    since_ms = int(one_year_ago.timestamp() * 1000)

    # Скачивание данных
    df_btc_usdt = fetch_ohlcv_data(symbol, timeframe, since_ms)

    if df_btc_usdt is not None:
        print("\nДанные успешно скачаны. Первые 5 строк:")
        print(df_btc_usdt.head())
        print("\nПоследние 5 строк:")
        print(df_btc_usdt.tail())
        print(f"\nРазмер DataFrame: {df_btc_usdt.shape}")
        
        # Сохранение данных в CSV-файл (опционально)
        # df_btc_usdt.to_csv('btc_usdt_1h.csv')
        # print("Данные сохранены в файл 'btc_usdt_1h.csv'")
(df_btc_usdt.shape,df_btc_usdt.tail)

Скачиваем данные для BTC/USDT с биржи binance...

Данные успешно скачаны. Первые 5 строк:
                         open      high       low     close       volume
timestamp                                                               
2022-09-23 18:00:00  18770.00  18834.42  18620.28  18635.02  21562.25099
2022-09-23 19:00:00  18636.12  18829.71  18620.31  18807.57  20617.96022
2022-09-23 20:00:00  18807.57  18956.85  18807.57  18852.17  14795.74700
2022-09-23 21:00:00  18853.46  19044.02  18803.00  19010.43   7749.80073
2022-09-23 22:00:00  19010.43  19341.09  18995.08  19333.18  17836.41446

Последние 5 строк:
                          open       high        low      close      volume
timestamp                                                                  
2025-09-22 13:00:00  112738.64  113148.29  112380.00  113130.63   813.72754
2025-09-22 14:00:00  113130.63  113432.76  112754.15  112933.16  1386.14648
2025-09-22 15:00:00  112933.16  113024.23  112576.42  112815.50   609.57711

((26279, 5),
 <bound method NDFrame.tail of                           open       high        low      close       volume
 timestamp                                                                   
 2022-09-23 18:00:00   18770.00   18834.42   18620.28   18635.02  21562.25099
 2022-09-23 19:00:00   18636.12   18829.71   18620.31   18807.57  20617.96022
 2022-09-23 20:00:00   18807.57   18956.85   18807.57   18852.17  14795.74700
 2022-09-23 21:00:00   18853.46   19044.02   18803.00   19010.43   7749.80073
 2022-09-23 22:00:00   19010.43   19341.09   18995.08   19333.18  17836.41446
 ...                        ...        ...        ...        ...          ...
 2025-09-22 13:00:00  112738.64  113148.29  112380.00  113130.63    813.72754
 2025-09-22 14:00:00  113130.63  113432.76  112754.15  112933.16   1386.14648
 2025-09-22 15:00:00  112933.16  113024.23  112576.42  112815.50    609.57711
 2025-09-22 16:00:00  112815.50  112888.00  112585.87  112600.68    359.00005
 2025-09-22 17:00:00 

In [3]:
df_btc_usdt.shape

(8760, 5)